## 1️⃣ Step 1: Set up Docker 

### 🔧 Key Docker Concepts

| **Concept**   | **Example**                        | **What It Means**                                         |
|---------------|------------------------------------|------------------------------------------------------------|
| **Image**     | `python:latest`, `postgres:15`     | A snapshot blueprint — like a cake recipe                  |
| **Container** | `docker run -it python:latest`     | A live instance of that image — like a baked cake          |
| **Dockerfile**| `FROM python:3.12 ...`             | A way to define your own image — ingredients + steps       |
| **Build**     | `docker build -t myimage .`        | Turn the Dockerfile into an image                          |
| **Run**       | `docker run -it myimage`           | Create a container from your image                         |
| **EntryPoint**| `--entrypoint=bash`                | Override the default "what to do when container starts"    |
| **Volume**    | `-v $(pwd):/app` (later)           | Mount your local files into container (for persistence)    |


### 🧠 Docker Mental Model: Typical Workflow

| **Step**                     | **Command**                                 | **What It Means**                                                                 |
| ---------------------------- | ------------------------------------------- | --------------------------------------------------------------------------------- |
| 🧾 Write a `Dockerfile`      | `FROM python:3.12`<br>`RUN pip install ...` | Define the environment you want (OS, packages, code, etc.)                        |
| 🏗️ Build the image          | `docker build -t my-image .`                | Create an image from your Dockerfile (bakes your "blueprint")                     |
| 🧪 Run a container           | `docker run -it my-image`                   | Start a container (your image, but running like a little isolated machine)        |
| 🔧 Do stuff in the container | `python`, `bash`, etc.                      | Run commands, test things, or run apps in that little machine                     |
| 🛑 Stop & remove when done   | `docker stop`, `docker rm`                  | Clean up your container when you’re finished                                      |
| 📦 (Optional) Mount volumes  | `-v $(pwd):/app`                            | Connect your local files to the container so changes are saved outside it         |
| 🌐 (Optional) Expose ports   | `-p 5432:5432`                              | Let your container be reachable on your machine (e.g., for Postgres, web servers) |


## Step 2: Run Postgres on Docker 

- Create a `docker-compose.yaml` file to define the Postgres service and credentials
- Use the **official Postgres image** from Docker Hub (`postgres:13`)
- Create a `ny_taxi_postgres_data/` folder to persist Postgres data outside the container